In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ! pip install datasets transformers

     |████████████████████████████████| 225kB 29.9MB/s 
     |████████████████████████████████| 2.3MB 45.7MB/s 
     |████████████████████████████████| 112kB 54.4MB/s 
     |████████████████████████████████| 245kB 51.0MB/s 
     |████████████████████████████████| 901kB 49.3MB/s 
     |████████████████████████████████| 3.3MB 52.7MB/s 


In [ ]:
from datasets import load_dataset
datapath = "/home/ckm/ck_project/data/sarcasm/"
datasets = load_dataset("csv", data_files={"train": datapath+'train.csv',"test":datapth+'test.csv',"valid": datapath+'val.csv'},encoding='utf-8')

Using custom data configuration default-a86a164abac00962


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-a86a164abac00962/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'Unnamed: 0.1', 'labels', 'tweets'],
        num_rows: 100000
    })
    test: Dataset({
        features: ['Unnamed: 0', 'Unnamed: 0.1', 'labels', 'tweets'],
        num_rows: 30000
    })
    valid: Dataset({
        features: ['Unnamed: 0', 'Unnamed: 0.1', 'labels', 'tweets'],
        num_rows: 36370
    })
})

In [ ]:
datasets=datasets.remove_columns('Unnamed: 0')

In [ ]:
model_checkpoint = "bert-base-multilingual-uncased"

In [ ]:
from transformers import BertForMaskedLM
model=BertForMaskedLM.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
!pip install transformers[sentencepiece]

     |████████████████████████████████| 1.1MB 4.2MB/s 


In [ ]:
from transformers import BertTokenizerFast
    
tokenizer = BertTokenizerFast.from_pretrained(model_checkpoint, use_fast=True)

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["tweets"],padding=True,truncation=True,max_length=128)

In [ ]:
datasets=datasets.remove_columns('Unnamed: 0.1')

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'tweets'],
        num_rows: 100000
    })
    test: Dataset({
        features: ['labels', 'tweets'],
        num_rows: 30000
    })
    valid: Dataset({
        features: ['labels', 'tweets'],
        num_rows: 36370
    })
})

In [ ]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=3)

In [ ]:
block_size = 128
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'token_type_ids', 'tweets'],
        num_rows: 100000
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'token_type_ids', 'tweets'],
        num_rows: 30000
    })
    valid: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'token_type_ids', 'tweets'],
        num_rows: 36370
    })
})

In [ ]:
tokenized_datasets=tokenized_datasets.remove_columns('labels')

In [ ]:
tokenized_datasets=tokenized_datasets.remove_columns('tweets')

In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=3,
)

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
import torch
device = torch.device('cuda:0')
model =model.to(device)

In [ ]:
training_args = TrainingArguments(
    "mbert",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=10,
    save_strategy = "epoch",
    save_total_limit = 2
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["valid"],
    data_collator=data_collator
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,4.047900,3.899386
2,3.745700,3.690484
3,3.660100,3.618581


TrainOutput(global_step=26088, training_loss=3.9436359294387175, metrics={'train_runtime': 13216.5847, 'train_samples_per_second': 1.974, 'total_flos': 0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 4096, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': -16044032, 'train_mem_gpu_alloc_delta': 2042150400, 'train_mem_cpu_peaked_delta': 359927808, 'train_mem_gpu_peaked_delta': 2633793536})

In [ ]:
import math
eval_results = trainer.evaluate(lm_datasets['test'])
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 36.60


In [ ]:
import os
output_dir ="/content/drive/MyDrive/Masked_language_bert"
output_model_file = os.path.join(output_dir, 'pytorch_model.bin')
output_config_file = os.path.join(output_dir, 'config.json')
model_to_save = model.module if hasattr(model, 'module') else model
torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(output_dir)

('/content/drive/MyDrive/Masked_language_bert/vocab.txt',)

In [ ]:
output_dir ="/content/drive/MyDrive/Masked_language_bert"

In [ ]:
from datasets import load_dataset

In [ ]:
from transformers import BertForSequenceClassification

In [ ]:
loaded_model=BertForSequenceClassification.from_pretrained(output_dir,num_labels=2)

Some weights of the model checkpoint at /content/drive/MyDrive/Masked_language_bert were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /

In [ ]:
from transformers import BertTokenizerFast

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained(output_dir) 

In [ ]:
dataset = load_dataset("csv", data_files={"train":'/content/drive/MyDrive/split_dataset/train.csv', "validation":'/content/drive/MyDrive/split_dataset/valid.csv',"test":'/content/drive/MyDrive/split_dataset/test.csv'})

Using custom data configuration default-3221b56a9807690e


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-3221b56a9807690e/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


In [ ]:
dataset=dataset.remove_columns('Unnamed: 0')

In [ ]:
dataset['train'][0]

{'labels': 0,
 'tweets': 'ather farouqui general secretary of ghar emphasis on preservation of of in his'}

In [ ]:
def preprocess_function(examples):
      return tokenizer(examples['tweets'], truncation=True,padding=True,max_length=128)

In [ ]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

In [ ]:
encoded_dataset=encoded_dataset.remove_columns('tweets')

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
import torch
device = torch.device('cuda:0')
loaded_model = loaded_model.to(device)
args = TrainingArguments(
    "FInetuned1",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
)

In [ ]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'token_type_ids'],
        num_rows: 115000
    })
    validation: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'token_type_ids'],
        num_rows: 21370
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'token_type_ids'],
        num_rows: 30000
    })
})

In [ ]:
args.place_model_on_device

True

In [ ]:
from sklearn.metrics import accuracy_score,label_ranking_average_precision_score,hamming_loss,f1_score,roc_auc_score

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    y_true=labels
    y_pred=predictions
    return accuracy_score(labels,predictions),f1_score(y_true, y_pred, average='micro'),roc_auc_score(y_true, y_pred, average='micro'),label_ranking_average_precision_score(y_true, y_pred),hamming_loss(y_true, y_pred)

In [ ]:
import numpy as np

In [ ]:
trainer = Trainer(
    loaded_model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,   
)

In [ ]:
import numpy as np

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.269900,0.278009,0.889892,0.882579
2,0.244700,0.281124,0.898830,0.894372
3,0.178100,0.324393,0.897894,0.896372


TrainOutput(global_step=21564, training_loss=0.24489936888759686, metrics={'train_runtime': 8852.4985, 'train_samples_per_second': 2.436, 'total_flos': 0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 0, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 299896832, 'train_mem_gpu_alloc_delta': 2012356608, 'train_mem_cpu_peaked_delta': 63004672, 'train_mem_gpu_peaked_delta': 1619138560})

In [ ]:
trainer.evaluate(encoded_dataset["test"])

In [ ]:
out='/content/drive/MyDrive/Final_bert_masked_model'
output_model_file = os.path.join(out, 'pytorch_model.bin')
output_config_file = os.path.join(out, 'config.json')
model_to_save = loaded_model.module if hasattr(loaded_model, 'module') else loaded_model
torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(out) 